# Assignment 1 -- Unit Commitment
## Optimisation
### Morgan Reilly, Shaurabh Kumar Singh 
### 20235398, (student number Shurabh)

In [2]:
from ortools.linear_solver import pywraplp

In [3]:
# Instantiate Solver
solver = pywraplp.Solver('Generator', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

### Decision Variables
* In a small country there are 10 generators of four types: Hydroelectric, Solid fuel, Gas, Solar.
* Each generator has a lower and upper bound on its production per hour (in MW/h). Between these bounds, production is not limited to discrete levels. 
* Each generator also has a cost for producing each MW, and produces a certain amount of CO2 per MW.

#### Optional Todo:
* Read this value in from csv file in form of loop?

In [4]:
# Create 10 variables with their lower and upper bounds
A = solver.NumVar(10, 100, 'A')  # hyrdo1
B = solver.NumVar(10, 80, 'B')  # hydro2
C = solver.NumVar(10, 60, 'C')  # hydro3
D = solver.NumVar(1, 10, 'D')  # hydro4
E = solver.NumVar(100, 900, 'E')  # solid1
F = solver.NumVar(100, 600, 'F')  # solid2
G = solver.NumVar(10, 100, 'G')  # solid 3
H = solver.NumVar(100, 400, 'H')  # gas1
I = solver.NumVar(0, 70, 'I')  # solar1
J = solver.NumVar(0, 20, 'J')  # solar2

### Constraints
* The maximum supply from any Solar generator depends on the time of day. Relative to the generator’s maximum, it can achieve 50% from 6am to 10am, 100% from 11am to 3pm, 50% from 4pm to 6pm, and 0% otherwise.
* The Solid fuel generators cannot change their amount of production from one hour to the next.

In [15]:
total_demand = 1461

solid_hours_const = E + F + G
solid_cost_const = 4.4*E + 4.4*F + 4.4*G

# Solar Constraints
# 12am - 6am & 6pm - 12am, at this point of time the power will be zero
solver.Add(I == J == 0, name="solar zeros")

# 6am - 10am -> 50% capacity
solver.Add(I <= 0.5*70)  # 50% of Max -> 70
solver.Add(J <= 0.5*20)  # 50% of Max -> 20

# 11am - 3pm -> 100% capacity
solver.Add(I <= 70)
solver.Add(J <= 20)

# All Generators
solver.Add(1.4*A + 1.4*B + 1.4*C + 1.4*D + 4.4*E + 4.4*F + 4.4*G
          + 9.1*H + 6.6*I + 6.6*J == total_demand, name="per hour constraint")



<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f9d0c5127b0> >

### Formaising Objective
* Goal is to meet demand at each hour at the minimum cost, by choosing how much energy each generator should produce per hour
* The demand from electricity customers varies by hour of the day. This is given in demand.csv. If we over-supply relative to demand, it can damage the electricity infrastructure. If we under-supply, then some customers can’t boil their kettles.

In [26]:
import pandas as pd

# Read in from generators.csv
generator_data = pd.read_csv("./generator_info.csv")

# Verify file load
generator_data.describe()

,lower_bound (MW),upper_bound (MW),cost/MW,CO2/MW
count,10.000000,10.000000,10.000000,10.000000
mean,34.100000,234.000000,4.110000,0.304600
std,45.671411,301.448356,2.727005,0.384678
min,0.000000,10.000000,1.400000,0.000000
25%,3.250000,62.500000,1.400000,0.024000
50%,10.000000,90.000000,4.400000,0.024000
75%,77.500000,325.000000,6.050000,0.737500
max,100.000000,900.000000,9.100000,0.820000


In [50]:
# Create the objective function
objective = solver.Objective()

# Rows, Cols from excel
# For 1 -> 10
# Multiply cost * power for A -> J
# Sum over all of this
# This should yeild minimum
costs = [] 
for index, row in generator_data.iterrows():
    # row[4] -> cost; row[5]
    costs.append(row[4])
    
objective.SetCoefficient(A, 15)
objective.SetCoefficient(B, 10)
objective.SetMinimization()

# Solve...
result = solver.Solve()
if result == solver.OPTIMAL:
    print_solution_and_sensitivity(solver, objective)
else:
    print("Problem is not feasible")

Problem is not feasible
